Importing the required libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

Loading the cifar10 dataset from keras dataset

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Data Normalization and pre-processing

In [3]:
#Normalizing the data to get data in range 0-1
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train = X_train / 255.0 
X_test = X_test / 255.0
#Converting to categorical 
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Splitting the training set into training and validation set

In [4]:
x_train, x_val, yy_train, yy_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Convolutional Neural Network model layers

In [5]:
model = Sequential()

#Convolutional 2D Layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
#Batch Normalization Layer
model.add(BatchNormalization())
#Max Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
#Dropout layer with a dropout rate of 0.25
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Flatten Layer
model.add(Flatten())
#Fully Connected Layer with relu activation function
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
#Output layer with softmax activation function
model.add(Dense(10, activation='softmax'))

Summary of our Model

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 64)       2

Compiling the model using categorical cross entropy loss function and adam optimizer.
Training the model for 20 epochs using a batch size of 32, also using our validation set as our validation dataset for model training 

In [6]:
met = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= met)
model.fit(x_train, yy_train, epochs=20 ,validation_data=(x_val, yy_val),batch_size = 32)

Epoch 1/20
1250/1250 [==============================] - 12s 7ms/step - loss: 1.6309 - accuracy: 0.4158 - precision: 0.6091 - recall: 0.2026 - val_loss: 1.4144 - val_accuracy: 0.4935 - val_precision: 0.6346 - val_recall: 0.3503
Epoch 2/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.2845 - accuracy: 0.5422 - precision: 0.7095 - recall: 0.3726 - val_loss: 1.5361 - val_accuracy: 0.5221 - val_precision: 0.6143 - val_recall: 0.4272
Epoch 3/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.1297 - accuracy: 0.5990 - precision: 0.7456 - recall: 0.4559 - val_loss: 1.1534 - val_accuracy: 0.6038 - val_precision: 0.7171 - val_recall: 0.5074
Epoch 4/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.0334 - accuracy: 0.6381 - precision: 0.7678 - recall: 0.5090 - val_loss: 1.3965 - val_accuracy: 0.5584 - val_precision: 0.6450 - val_recall: 0.4750
Epoch 5/20
1250/1250 [==============================] - 8s 6ms/step - loss: 0.9587 - accuracy: 

Evaluating model performance on our test set

In [7]:
evaluate = model.evaluate(X_test, y_test)
print("Accuracy: ",evaluate[1])
print("Precision: ", evaluate[2])
print("Recall: ", evaluate[3])

313/313 [==============================] - 1s 4ms/step - loss: 0.6364 - accuracy: 0.7938 - precision: 0.8417 - recall: 0.7562
Accuracy:  0.7937999963760376
Precision:  0.8417186141014099
Recall:  0.7562000155448914


Reshaping our testing and training features as we need a 2d matrix for our KNN and Naive Bayes ML models

In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

Training and evaluating KNN classifier performance on cifar10

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
predict = knn.predict(X_test)
print("KNN classifier")
print(f"Accuracy: {accuracy_score(y_test, predict)}")
print("Precision: ", precision_score(y_test, predict,average='weighted'))
print("Recall: ", recall_score(y_test, predict,average='weighted'))

KNN classifier
Accuracy: 0.2453
Precision:  0.5645490388999024
Recall:  0.2453


We have a 2d array so we'll reshape to a 1d array as in input to our Naive Bayes Classifier

In [10]:
yy_train = y_train[:, 0]
yy_test = y_test[:, 0]

Training and evaluating Naive Bayes classifier performance on cifar10

In [11]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, yy_train)
predict = gnb.predict(X_test)
print("KNN classifier")
print(f"Accuracy: {accuracy_score(yy_test, predict)}")
print("Precision: ", precision_score(yy_test, predict,average='weighted'))
print("Recall: ", recall_score(yy_test, predict,average='weighted'))

KNN classifier
Accuracy: 0.6967
Precision:  0.8814086553491397
Recall:  0.6967


**Comparitive Analysis**

*Convolutional Neural Network*

CNN model takes longer time in training however once trained it provides the best test time perdictions. We have trained our CNN model for 20 epochs with a 32 batch size of input data. Our model has 3 each of our Conv2D, Batch Normalization and Max2D Pooling Layers. Additionally we have used a dropout of 0.25 so that at 75% of neurons only contribute at one time giving a chance to all the neurons to participate. We have used relu activation function at our fully connected layer and a softmax activation function at our output layer which predicts the labels for 10 classes of cifar10 dataset. Our model achieves a test time accuracy of 79.3% followed by a recall and precision of 0.75 and 0.84 respectively.

*K Nearest Neighbor*

We have trained our KNN model for 5 neighbors on cifar10 training dataset. We have further evaluated our models performance for the 10,000 test dataset items. Our model does'nt produce satisfactory results. Model produces an accuracy, precision and recall of 24.5%,0.56,0.24 respectively.

*Naive Bayes Classifier*

Our Naive Bayes Classifier produces better results than KNN however not as good as our CNN model. Our model achieves a test time accuracy of 69.6% followed by precision and recall of 0.88,0.69 respectively.


**Comparison**

In terms of time complexity, CNN took the longest time at training followed by KNN, however Naive Bayes took the least time. In terms of performance as we can see from the evaluation metrics CNN performed the best in classification of cifar-10 dataset. All these models can be used for mutliclass classification but as we can see CNN is a neural network where at each layers features and weights are learned it produces better results as compared to others